In [ ]:
# import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import time
from datetime import datetime

# import the requests library
import requests

# import the api key
from config import weather_api_key

In [ ]:
# create a set of random latitude and longitude combinations
lats = np.random.uniform(-90, 90, size=2000)
lngs = np.random.uniform(-180, 180, size=2000)
lat_lngs = zip(lats, lngs)
# add the lat and lngs to a list
coordinates = list(lat_lngs)

In [ ]:
# create a list for holding the cities
cities = []

# identify the nearest city for each latitude and longitude combination
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # if the city is unique, add it to the cities list
    if city not in cities:
        cities.append(city)
# print the city count to confirm sufficient count
len(cities)

In [ ]:
url = f'http://api.openweathermap.org/data/2.5/weather?units=Imperial&appid={weather_api_key}'

# create an empty list to hold the weather data
city_data = []

# print the beginning of the logging
print('Beginning Data Retrieval     ')
print('-----------------------------')

# create counters
record_count = 1
set_count = 1

# loop through all the cities in our list
for i, city in enumerate(cities):
    
    #group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)
        
    # create endpoint url with each city
    city_url = url + '&q=' + city.replace(' ', '+')
    
    # log the url, record, and set numbers and the city
    print(f'Processing Record {record_count} of Set {set_count} | {city}')
    
    # add 1 to the record count
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather['weather'][0]['description']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                         'Current Description':city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
# convert the array of distionaries to a df
city_data_df = pd.DataFrame(city_data)

# reorder the columns
new_column_order = ['City', 'Country', 'Date', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',  'Wind Speed', 'Current Description']
city_data_df = city_data_df[new_column_order]

# show the df
city_data_df.head()

In [ ]:
# create the output file (CSV)
output_data_file = 'WeatherPy_Database.csv'

# export the city_data into a csv
city_data_df.to_csv(output_data_file, index_label = 'City_ID')